In [18]:
import os
import math
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
from PIL import Image
from torchvision.transforms import v2

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define Dataset Class
class WasteDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.images = []
        self.labels = []
        self.class_names = sorted(os.listdir(folder_path))

        for label, class_name in enumerate(self.class_names):
            class_path = os.path.join(folder_path, class_name)
            image_files = [file for file in os.listdir(class_path) if file.endswith('.jpg')]

            for file in image_files:
                self.images.append(os.path.join(class_path, file))
                self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]

        # Open image safely
        with open(img_path, 'rb') as f:
            image = Image.open(f).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long)

# Define Transformations
train_transform = v2.Compose([
    v2.Resize(256),
    v2.RandomResizedCrop(224),
    v2.RandomHorizontalFlip(p=0.5),
    v2.ColorJitter(0.3, 0.3, 0.3, 0.1),
    v2.ToTensor(),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = v2.Compose([
    v2.Resize(224),
    v2.CenterCrop(224),
    v2.ToTensor(),
    v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Dataset Paths
dataset_path = "/home/jovyan/Waste_Model/wastes1"

# Load Dataset
full_dataset = WasteDataset(os.path.join(dataset_path, 'train'), transform=train_transform)
test_dataset = WasteDataset(os.path.join(dataset_path, 'test'), transform=test_transform)

# Split into train (80%) and validation (20%)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(
    train_dataset,
    batch_size=128,
    shuffle=True,
    num_workers=2,
    pin_memory=True,
)

val_loader = DataLoader(
    val_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=2,
    pin_memory=True,
)

test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=2,
    pin_memory=True,
)

# Neural Network Model
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)

        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)  # Adaptive pooling
        self.flatten = nn.Flatten()

        self.dropout1 = nn.Dropout(p=0.3)

        self.linear1 = nn.Linear(256, 512)
        self.linear2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)

        x = self.relu(self.bn3(self.conv3(x)))
        x = self.pool(x)

        x = self.relu(self.bn4(self.conv4(x)))
        x = self.pool(x)

        x = self.global_avg_pool(x)
        x = self.flatten(x)
        x = self.relu(self.linear1(x))
        x = self.dropout1(x)
        x = self.linear2(x)

        return x

# Define save path
save_path = ""

# Initialize Model
model = NeuralNet().to(device)

# Load the last saved model if it exists
if os.path.exists(save_path):
    model.load_state_dict(torch.load(save_path, weights_only=True))
    model.train()  # Set model back to training mode
    print("Loaded model")
else:
    print("No saved model")


# Training Function
def train_model(model, train_loader, val_loader, loss_fn, optimizer, num_epochs=10, save_path="models/waste_classification.pth"):
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    model.to(device)
    best_val_acc = 0.0
    train_losses, val_losses = [], []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            labels = labels.long()
            loss = loss_fn(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # Validation
        model.eval()
        val_loss, correct, total = 0.0, 0, 0
        with torch.no_grad():
            for val_inputs, val_labels in val_loader:
                val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
                val_outputs = model(val_inputs)

                loss = loss_fn(val_outputs, val_labels.long())
                val_loss += loss.item()

                _, predicted = torch.max(val_outputs, 1)
                total += val_labels.size(0)
                correct += (predicted == val_labels).sum().item()

        avg_val_loss = val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        val_acc = 100 * correct / total

        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_acc:.2f}%")

        # Save model per epoch
        model_save_path = f"{save_path}_epoch_{epoch+1}.pth"
        torch.save(model.state_dict(), model_save_path)
        print(f"Model saved at {model_save_path}")

    return model, train_losses, val_losses

# Test Model Function
def test_model(model, test_loader):
    model.to(device)
    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy

# Training Configuration
NUM_EPOCHS = 20
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-5)

# Train Model
trained_model, train_loss, val_loss = train_model(model, train_loader, val_loader, loss_fn, optimizer, num_epochs=NUM_EPOCHS, save_path="models/waste_classification.pth")

# Test Model
test_accuracy = test_model(trained_model, test_loader)


Using device: cuda
No saved model found. Training from scratch.
Epoch [1/20], Train Loss: 0.5186, Validation Loss: 0.4366, Validation Accuracy: 80.61%
Model saved at models/waste_classification.pth_epoch_1.pth
Epoch [2/20], Train Loss: 0.4209, Validation Loss: 0.4271, Validation Accuracy: 80.63%
Model saved at models/waste_classification.pth_epoch_2.pth
Epoch [3/20], Train Loss: 0.3955, Validation Loss: 0.4038, Validation Accuracy: 81.96%
Model saved at models/waste_classification.pth_epoch_3.pth
Epoch [4/20], Train Loss: 0.3872, Validation Loss: 0.3808, Validation Accuracy: 83.60%
Model saved at models/waste_classification.pth_epoch_4.pth
Epoch [5/20], Train Loss: 0.3892, Validation Loss: 0.3766, Validation Accuracy: 84.18%
Model saved at models/waste_classification.pth_epoch_5.pth
Epoch [6/20], Train Loss: 0.3744, Validation Loss: 0.3659, Validation Accuracy: 83.56%
Model saved at models/waste_classification.pth_epoch_6.pth
Epoch [7/20], Train Loss: 0.3629, Validation Loss: 0.3461, V

In [20]:
# Give Model Our own images
model = NeuralNet().to(device)
model.load_state_dict(torch.load("/home/jovyan/Waste_Model/models/waste_classification.pth_epoch_20.pth"))
model.eval()  # Set to evaluation mode


def predict_image(image_path, model, class_names, device="cpu"):
    # Load and preprocess image
    with Image.open(image_path) as img:
        image = img.convert("RGB")

    transform = v2.Compose([
        v2.Resize(224),
        v2.CenterCrop(224),
        v2.ToTensor(),
        v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    image_tensor = transform(image).unsqueeze(0).to(device)  # Add batch dimension
    
    # Make prediction
    with torch.no_grad():
        output = model(image_tensor)
        probabilities = torch.nn.functional.softmax(output[0], dim=0)
        conf, class_idx = torch.max(probabilities, 0)
    
    return class_names[class_idx.item()], conf.item()

#usage
image_path = "/home/jovyan/Waste_Model/Own_images/cokebottle.jpg"  # Replace with your image path
class_names = full_dataset.class_names  # Get class names from your dataset

pred_class, confidence = predict_image(image_path, model, class_names, device)
print(f"Predicted: {pred_class} (Confidence: {confidence:.2%})")
    

Predicted: Recycle (Confidence: 92.37%)


/tmp/ipykernel_103/863074082.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/home/jovyan/Waste_Model/models/waste_classification.pth_e

In [ ]:
!nvidia-smi